In [1]:
class Moon:
    
    def __init__(self, name, x, y, z, vx=0, vy=0, vz=0):
        self.name = name
        self.x, self.y, self.z = x, y, z
        self.vx, self.vy, self.vz = vz, vy, vz
    
    def velocity(self, moon):
        self.vx += -1 if self.x > moon.x else 1 if self.x < moon.x else 0
        self.vy += -1 if self.y > moon.y else 1 if self.y < moon.y else 0
        self.vz += -1 if self.z > moon.z else 1 if self.z < moon.z else 0
    
    def apply(self):
        self.x += self.vx
        self.y += self.vy
        self.z += self.vz
    
    def __eq__(self, moon):
        return self.name == moon.name
    
    def __str__(self):
        return (f"[{self.name:<8}] x={self.x:>3}, y={self.y:>3}, z={self.z:>3} | "
                f"vx={self.vx:>3}, vy={self.vy:>3}, vz={self.vz:>3} | energy={self.energy:>3}")
    
    @property
    def energy(self):
        return (
            sum((abs(self.x), abs(self.y), abs(self.z))) *
            sum((abs(self.vx), abs(self.vy), abs(self.vz))))
    
    @property
    def data(self):
        return dict(
            name=self.name, x=self.x, y=self.y, z=self.z, 
            vx=self.vx, vy=self.vy, vz=self.vz)
    

class System:
    
    def __init__(self, *moons, step=0):
        self.moons = moons
        self.step = step
        
    def cycle(self, until=None):
        until = until if until else self.step + 1
        while self.step < until:
            self.step += 1  
            for moon in self.moons:
                for other in self.moons:
                    if moon == other:
                        continue
                    moon.velocity(other)
            for moon in self.moons:
                moon.apply()
        return self
    
    def __str__(self):
        return '\n'.join([
            f"Step: {self.step:>04}",
            '\n'.join(str(moon) for moon in self.moons),
            f"Total energy: {self.energy:>04}"])

    def __repr__(self):
        return str(self)
    
    @property
    def energy(self):
        return sum(moon.energy for moon in self.moons)
    
    @property
    def values(self):
        return {coord: tuple(
            (getattr(moon, coord), getattr(moon, f'v{coord}')) for moon in self.moons
        ) for coord in ('x', 'y', 'z')}
    
    @property
    def copy(self):
        return System(*(Moon(**moon.data) for moon in self.moons), step=self.step)

In [2]:
system = System(
    Moon(name="Io", x=4, y=12, z=13),
    Moon(name="Europe", x=-9, y=14, z=-3),
    Moon(name="Ganymede", x=-7, y=-1, z=2),
    Moon(name="Callisto", x=-11, y=17, z=-1),
)

In [3]:
system.copy.cycle(1000)

Step: 1000
[Io      ] x=  1, y=-47, z=-40 | vx= -1, vy=-17, vz= 12 | energy=2640
[Europe  ] x=-86, y= 65, z= 34 | vx=  0, vy=  2, vz= -5 | energy=1295
[Ganymede] x=  7, y= -5, z=  3 | vx= -8, vy= 15, vz= -6 | energy=435
[Callisto] x= 55, y= 29, z= 14 | vx=  9, vy=  0, vz= -1 | energy=980
Total energy: 5350

In [4]:
from functools import reduce
from math import gcd

def lcms(*numbers):
    def lcm(a, b):
        return abs(a * b) // gcd(a, b)
    return reduce(lcm, numbers)

In [5]:
def find_cycle(system):
    loop = 0
    history = dict(x=set(), y=set(), z=set())
    loops = dict(x=0, y=0, z=0)
    while not all(loops.values()):
        system.cycle()
        for coord, value in system.values.items():
            if loops[coord]:
                continue
            if value in history[coord]:
                loops[coord] = loop
            history[coord].add(value)
        loop += 1
    return lcms(*loops.values())

In [6]:
find_cycle(system.copy)

467034091553512